In [1]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
conn = sqlite3.connect("../db.sqlite3")

print("Open Database successfully")
cur = conn.cursor()
cur.execute("PRAGMA table_info(grade_info_gradeinfo)")
cur.fetchall()

Open Database successfully
[(0, 'id', 'integer', 1, None, 1), (1, 'credit', 'decimal', 1, None, 0), (2, 'grade', 'decimal', 0, None, 0), (3, 'ifRestudied', 'bool', 1, None, 0), (4, 'testKind', 'varchar(2)', 1, None, 0), (5, 'optionChoice', 'varchar(2)', 1, None, 0), (6, 'LessonID_id', 'integer', 1, None, 0), (7, 'SchoolYearID_id', 'integer', 1, None, 0), (8, 'user_id', 'integer', 1, None, 0)]


In [4]:
cur.execute("select count(*) from grade_info_gradeinfo where user_id in (select id from account_user where isPostgraduate = True)").fetchall()

[(19170,)]

In [15]:
cur.execute("select * from grade_info_gradeinfo where user_id in (select id from account_user where isPostgraduate = True)")

In [16]:
result = cur.fetchall()

In [17]:
result[0]

(347587, 2, 92, 0, 'np', 'uc', 2700001, 5, 2991)

In [46]:
classification_mapping = {
    "专业选修课": "po",
    "专业必修课": "pm",
    "选修课": "op",
    "未分类课": "uc",
    "公共必修课": "cm"
}

In [19]:
def unClassified(n, *args):
    if n[6] == classification_mapping[args]:
        return n

In [23]:
filter(unClassified, result)

In [53]:
result_po = [i for i in result if i[5] == classification_mapping["专业选修课"]]
result_pm = [i for i in result if i[5] == classification_mapping["专业必修课"]]
result_op = [i for i in result if i[5] == classification_mapping["选修课"]]
result_cm = [i for i in result if i[5] == classification_mapping["公共必修课"]]
result_uc = [i for i in result if i[5] == classification_mapping["未分类课"]]

In [3]:
user = cur.execute("select id from account_user where isPostgraduate = True").fetchall()

In [97]:
len(user)

1469

In [11]:
user = [u[0] for u in user]

In [13]:
pd_grade = pd.DataFrame(columns=["user_id", "专业选修课成绩", "专业选修课学分", "专业必修课成绩", "专业必修课学分", "选修课成绩", "选修课学分", "公共必修课成绩", "公共必修课学分", "未分类课成绩", "未分类课学分", "总成绩", "总学分"])

In [19]:
count = 0
for user_id in user:
    dic1 = {
        "credit_pm": 0,
        "credit_op": 0,
        "credit_po": 0,
        "credit_cm": 0,
        "credit_uc": 0,
        "average_grade_pm": 0,
        "average_grade_po": 0,
        "average_grade_op": 0,
        "average_grade_cm": 0,
        "average_grade_uc": 0,
    }
    for grade in result:
        if grade[8] == user_id:
            dic1["credit_" + grade[5]] += grade[1]
            dic1["average_grade_" + grade[5]] += grade[2]*grade[1]
    lis = ["pm", "op", "po", "cm", "uc"]
    total_grade = 0
    total_credit = 0
    for item in lis:
        total_grade += dic1["average_grade_" + item]
        if dic1["credit_" + item] != 0:
            dic1["average_grade_" + item] /= dic1["credit_" + item]
        total_credit += dic1["credit_" + item]
    if total_credit != 0:
        total_grade /= total_credit
    pd_grade.loc[count] = [user_id, dic1["average_grade_po"], dic1["credit_po"], 
                           dic1["average_grade_pm"], dic1["credit_pm"],
                           dic1["average_grade_op"], dic1["credit_op"], 
                           dic1["average_grade_cm"], dic1["credit_cm"],
                           dic1["average_grade_uc"], dic1["credit_uc"],
                           total_grade, total_credit
                          ]
    count += 1

In [20]:
pd_grade

,user_id,专业选修课成绩,专业选修课学分,专业必修课成绩,专业必修课学分,选修课成绩,选修课学分,公共必修课成绩,公共必修课学分,未分类课成绩,未分类课学分,总成绩,总学分
0,2991.0,0.0,0.0,0.0,0.0,90.461538,6.5,0.0,0.0,83.378378,18.5,85.220000,25.0
1,2992.0,0.0,0.0,0.0,0.0,89.250000,8.0,0.0,0.0,89.529412,17.0,89.440000,25.0
2,2993.0,0.0,0.0,0.0,0.0,90.947368,9.5,0.0,0.0,85.903226,15.5,87.820000,25.0
3,2994.0,0.0,0.0,0.0,0.0,85.818182,11.0,0.0,0.0,89.785714,14.0,88.040000,25.0
4,2995.0,0.0,0.0,0.0,0.0,82.692308,6.5,0.0,0.0,86.459459,18.5,85.480000,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1464,4455.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,4.000000,3.0,4.000000,3.0
1465,4456.0,93.0,2.0,0.0,0.0,0.000000,0.0,85.0,2.0,85.714286,7.0,86.909091,11.0
1466,4457.0,0.0,0.0,0.0,0.0,0.000000,0.0,85.0,2.0,79.900000,10.0,80.750000,12.0
1467,4458.0,81.0,2.0,0.0,0.0,0.000000,0.0,0.0,0.0,84.333333,9.0,83.727273,11.0


In [120]:
len(pd_grade)

1469

In [21]:
pd_grade.sort_values("user_id").reset_index(drop=True).to_csv("grade_info_2.csv")